<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       MAMean Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial'><b>MAMean</b></p>
<p style = 'font-size:16px;font-family:Arial'> The MAMean() function forecasts a user-defined number of periods into the future, that is the number of periods beyond the last observed sample point in the series.</p>

<p style = 'font-size:16px;font-family:Arial'>The MAMean function uses available historical data to forecast activity h periods ahead. It is classified as a flat forecaster because given t historical observations, the forecast for any period beyond t+1 is equal to the forecast for t+1. Another use of this forecaster is to forecast one-period ahead. If historical data is available up to and including time t, then it can be used to forecast the value at time t+1. When the actual time t+1 value becomes available, it can be used to forecast the value at time t+2, and so on.</p>
 


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    MAMean,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_MAMean_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
 load_example_data("uaf", ["orders1"])

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
df = DataFrame.from_table("orders1")
df

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. MAMean</b>
<p style = 'font-size:16px;font-family:Arial'>The MAMean() function forecasts a user-defined number of periods into the future, that is the number of periods beyond the last observed sample point in the series.</p>

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the convolve function.</p>

In [ ]:
result = TDSeries(data=df,
                      id="store_id",
                      row_index="seq",
                      row_index_style="SEQUENCE",
                      payload_field="sales",
                      payload_content="REAL") 

<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(MAMean)

<p style = 'font-size:16px;font-family:Arial'> We will forecast 8 number of periods into the future beyond the last observed sample point in the series using the moving average algorithm and generate metadata as well as residuals data.</p>

<p style = 'font-size:16px;font-family:Arial'>Input Parameters:</p>
<li style = 'font-size:16px;font-family:Arial'><code>data:</code> Specifies a logical univariate series with historical data.</li>
<li style = 'font-size:16px;font-family:Arial'><code>forecast_periods:</code> Specifies the number of periods to forecast ahead. The argument "forecast_periods" must be a positive integer value in the range [1, 32000]</li>
<li style = 'font-size:16px;font-family:Arial'><code>algorithm:</code> Specifies the type of algorithm to use. Permitted Values:
    <b>MA</b>, <b>MEAN</b> , <b>NAIVE</b></li>
<li style = 'font-size:16px;font-family:Arial'><code>prediction_intervals:</code> Specifies the confidence level for the prediction, such that 85 means 85% confidence.  Permitted Values: <b>NONE<b>, <b>80</b>, <b>95</b>, <b>BOTH</b></li>    
<li style = 'font-size:16px;font-family:Arial'><code>k_order:</code> Specifies the moving average forecast. The argument "k_order" must be a positive int value in the range [1, 32000]</li>
<li style = 'font-size:16px;font-family:Arial'><code>fit_metrics:</code> Specifies a flag to generate the secondary result set that contains the model metadata statistics. When set to True, function generate the secondary result set, otherwise not. The generated result set can be retrieved using the attribute fitmetadata of the function output.</li>
<li style = 'font-size:16px;font-family:Arial'><code>residuals:</code> Specifies a flag to generate the tertiary result set that contains the model residuals. When set to True, means generate the tertiary result set, otherwise not. The generated result set can be retrieved using the attribute fitresiduals of the function output.</li>

In [ ]:
uaf_out = MAMean(data=result,
                 forecast_periods=8,
                 algorithm='MA',
                 prediction_intervals='BOTH',
                 k_order=3,
                 fit_metrics=True,
                 residuals=True)
mamean_df = uaf_out.result
mamean_df

<p style = 'font-size:16px;font-family:Arial'><code>fitmetadata</code> - Available when "model_stats" is set to True, otherwise not.</p>

In [ ]:
uaf_out.fitmetadata

<p style = 'font-size:16px;font-family:Arial'><code>fitresiduals </code> - Available when "residuals" is set to True, otherwise not.</p>

In [ ]:
uaf_out.fitresiduals

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
db_drop_table('orders1')

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>MAMean function reference: <a href = 'https://docs.teradata.com/search/all?query=MAMean&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>